# notebook for create init and true test model

In [9]:
import numpy as np
import math

# grid
R_earth = 6371.0

rr1=5900 
rr2=6400
tt1=(30.0)/180*math.pi
tt2=(50.0)/180*math.pi
pp1=(15.0)/180*math.pi
pp2=(40.0)/180*math.pi

# source
r0=(rr1+rr2)/2.0+0.01
t0=(tt1+tt2)/2.0+0.0001
p0=(pp1+pp2)/2.0+0.0001

z0 = r0 * math.sin(t0)
x0 = r0 * math.cos(t0) * math.cos(p0)
y0 = r0 * math.cos(t0) * math.sin(p0)



In [10]:
def linear_velocity_time(x,y,z):
    c0=7.0
    gx=-0.01*x0/math.sqrt(x0**2+y0**2+z0**2)
    gy=-0.01*y0/math.sqrt(x0**2+y0**2+z0**2)
    gz=-0.01*z0/math.sqrt(x0**2+y0**2+z0**2)
    vel = c0 + gx*(x-x0) +gy*(y-y0) + gz*(z-z0)

    normg = math.sqrt(gx**2+gy**2+gz**2)
    dis = math.sqrt((x-x0)**2+(y-y0)**2+(z-z0)**2)
    arcz = 1+0.5*(1.0/vel)*(1.0/c0)*normg**2*dis**2
    arccosh = math.log(arcz+math.sqrt(arcz**2-1))

    time = arccosh/normg

    return [vel,time]

In [11]:
import h5py
import os

try:
    os.mkdir("models")
except:
    print("dir models exists")

for Ngrid in [41,61,81,121,161]:
    # model
    n_rtp = [Ngrid,Ngrid,Ngrid]
    dr = (rr2-rr1)/(n_rtp[0]-1)
    dt = (tt2-tt1)/(n_rtp[1]-1)
    dp = (pp2-pp1)/(n_rtp[2]-1)
    rr = np.array([rr1 + x*dr for x in range(n_rtp[0])])
    tt = np.array([tt1 + x*dt for x in range(n_rtp[1])])
    pp = np.array([pp1 + x*dp for x in range(n_rtp[2])])

    eta_init = np.zeros(n_rtp)
    xi_init  = np.zeros(n_rtp)
    zeta_init = np.zeros(n_rtp)
    fun_init = np.zeros(n_rtp)
    vel_init = np.zeros(n_rtp)
    solution = np.zeros(n_rtp)

    for ir in range(n_rtp[0]):
        for it in range(n_rtp[1]):
            for ip in range(n_rtp[2]):

                z = rr[ir] * math.sin(tt[it])
                x = rr[ir] * math.cos(tt[it]) * math.cos(pp[ip])
                y = rr[ir] * math.cos(tt[it]) * math.sin(pp[ip])

                [vel,time] = linear_velocity_time(x,y,z)
                vel_init[ir,it,ip] = vel
                solution[ir,it,ip] = time

    # write out in hdf5 format

    fout_init = h5py.File('models/model_N%d_%d_%d.h5'%(n_rtp[0],n_rtp[1],n_rtp[2]), 'w')
    # write out the arrays eta_init, xi_init, zeta_init, fun_init, a_init, b_init, c_init, f_init
    fout_init.create_dataset('eta', data=eta_init)
    fout_init.create_dataset('xi', data=xi_init)
    fout_init.create_dataset('zeta', data=zeta_init)
    fout_init.create_dataset('vel', data=vel_init)
    fout_init.close()

    fout_solution = h5py.File('models/solution_N%d_%d_%d.h5'%(n_rtp[0],n_rtp[1],n_rtp[2]), 'w')
    # write out the arrays eta_init, xi_init, zeta_init, fun_init, a_init, b_init, c_init, f_init
    group = fout_solution.create_group('src_s0')
    group.create_dataset('T_res', data=solution)
    fout_solution.close()


dir models exists


# prepare src station file

The following code creates a src_rec_file for the inversion, which describes the source and receiver positions and arrival times.
Format is as follows:

```
        26 1992  1  1  2 43  56.900    1.8000     98.9000 137.00  2.80    8    305644 <- src   　: id_src year month day hour min sec lat lon dep_km mag num_recs id_event
     26      1      PCBI       1.8900     98.9253   1000.0000  P   10.40  18.000      <- arrival : id_src id_rec name_rec lat lon elevation_m phase epicentral_distance_km arrival_time_sec
     26      2      MRPI       1.6125     99.3172   1100.0000  P   50.84  19.400
     26      3      HUTI       2.3153     98.9711   1600.0000  P   57.84  19.200
     ....

```

In [12]:

# dummys
src_id      = 0
src_year    = 1998
src_month   = 1
src_day     = 1
src_hour    = 0
src_minute  = 0
src_sec     = 0
src_lat     = t0 / math.pi * 180
src_lon     = p0 / math.pi * 180
src_dep     = R_earth - r0
src_mag     = 3.0
src_name    = "s0"


Nr = 10; Nt = 10; Np = 10
N_rec       = Nr * Nt * Np



fname = 'src_rec_true.dat'
doc_src_rec = open(fname,'w')

doc_src_rec.write('%8d %8d %2d %2d %2d %2d %5.2f %9.4f %9.4f %8.4f %5.2f %5d %s\n'%
                 (src_id,src_year,src_month,src_day,src_hour,src_minute,src_sec,src_lat,src_lon,src_dep,src_mag,N_rec,src_name))

count = 0
for i in range(Nr):
    for j in range(Nt):
        for k in range(Np):
            rec_id      = count
            rec_name    = "r%d"%(count)
            rec_lat     = 32 + (48-32)/(Nt-1) * j   
            rec_lon     = 17 + (38-17)/(Np-1) * k   
            rec_dep     = (0 + 300/(Nr-1) * i)
            rec_ele     = -rec_dep*1000
            phase       = "P"

            r = R_earth - rec_dep
            t = rec_lat/180.0*math.pi
            p = rec_lon/180.0*math.pi
            z = r * math.sin(t)
            x = r * math.cos(t) * math.cos(p)
            y = r * math.cos(t) * math.sin(p)
            vel,time = linear_velocity_time(x,y,z)

            doc_src_rec.write('%8d %8d %6s %9.4f %9.4f %9.1f %s %8.3f\n'%(src_id,rec_id,rec_name,rec_lat,rec_lon,rec_ele,phase,time))

            count+=1
doc_src_rec.close()



        